# BOSTON

I have chosen this city because I'm going to travel there next summer. 
I downloaded the osm-file here:
https://mapzen.com/data/metro-extracts/metro/boston_massachusetts/

### Import libraries

In [10]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import json
import schema
import csv
import cerberus
import pandas as pd
import sqlalchemy
import os

### Filename

In [3]:
dataset="boston_massachusetts.osm"

## Auditing and Cleaning the Dataset

### Counting Tags

We parse the map file to find out what tags are there and how many

In [4]:
def count_tags(dataset):
    tags={}
    for event,element in ET.iterparse(dataset):
        if element.tag in tags:
            tags[element.tag]+=1
        else:
            tags[element.tag]=1
    return tags

In [5]:
count_tags(dataset)

{'bounds': 1,
 'member': 10805,
 'nd': 2321860,
 'node': 1930240,
 'osm': 1,
 'relation': 1248,
 'tag': 896347,
 'way': 308723}

### Key Types

Here we check the "k" value for each tag and see if there are any potential problems

In [6]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.match(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.match(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.match(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


In [7]:
process_map(dataset)

{'lower': 787959, 'lower_colon': 68370, 'other': 40018, 'problemchars': 0}

### Exploring Users

Here we find out how many unique users have contributed to the map

In [8]:
def get_user(element):
       
    return element.get("uid")
    
def process_users(dataset):
    users = []
    for _, element in ET.iterparse(dataset):
        if element.tag=="node" or element.tag=="way" or element.tag=="relation":
            val=get_user(element)
            if val in users:
                continue
            else:
                users.append(val)
   
    return users



In [9]:
users=process_users(dataset)
users[0:7]

['326503', '8609', '14293', '3983283', '443130', '165061', '7906']

### Improving Street Names

The main problem in the map is the street types. They are often just abbreviations, uppercase words or lowercase word. We audit the OSM FILE and change the variable 'mapping' to reflect the changes needed to fix the unexpected street types to the appropriate ones in the expected list.

In [11]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Court", "Place", "Square", "Lane","Trail", "Parkway", "Commons","Way",
            "Alley","Steeg","Avenue","Laan","Boulevard","Kringweg","Close","Crescent","Singel",
            "Drive","Rylaan","Place","Oord","Road","Weg","Street","Straat"]


mapping = { "St": "Street",
           "ST": "Street",
            "st": "Street",
           "st.": "Street",
           "st,": "Street",
            "street":"Street",
           'Sq.': 'Square',
            "Ave": "Avenue",
            "ave": "Avenue",
           'Ave.': 'Avenue',
            "Rd.": "Road",
           "Rd": "Road",
            "Cresent":"Crescent",
            "drive":"Drive",
           'HIghway': 'Highway',
           'Hwy': 'Highway',
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                    tag.attrib['v']=update_name(tag.attrib['v'], mapping)    
    return street_types

def update_name(name, mapping):
    for street_type in mapping:
        if street_type in name:
            name = re.sub(r'\b' + street_type+ r'\b\.?', mapping[street_type],name)
    return name

In [12]:
audit(dataset)

defaultdict(set,
            {'1100': {'First Street, Suite 1100'},
             '1702': {'Franklin Street, Suite 1702'},
             '3': {'Kendall Square - 3'},
             '303': {'First Street, Suite 303'},
             '4': {'Pier 4'},
             '6': {'South Station, near Track 6'},
             '846028': {'PO Box 846028'},
             'Artery': {'Southern Artery'},
             'Ave': {'738 Commonwealth Ave',
              'Blue Hill Ave',
              'Boston Ave',
              'College Ave',
              'Commonwealth Ave',
              'Concord Ave',
              'Everett Ave',
              'Francesca Ave',
              'Harrison Ave',
              'Highland Ave',
              'Josephine Ave',
              'Lexington Ave',
              'Massachusetts Ave',
              'Morrison Ave',
              'Mystic Ave',
              'Sagamore Ave',
              'Somerville Ave',
              "St. Paul's Ave",
              'Washington Ave',
              'Western 

### Saving csv files

Our task is to prepare the data to be inserted into a SQL database. To do so we will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files. These csv files can then easily be imported to a SQL database as tables.

In [13]:
OSM_PATH = "example.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def load_new_tag(element, secondary, default_tag_type):
    """
    Load a new tag dict to go into the list of dicts for way_tags, node_tags
    """
    new = {}
    new['id'] = element.attrib['id']
    if ":" not in secondary.attrib['k']:
        new['key'] = secondary.attrib['k']
        new['type'] = default_tag_type
    else:
        post_colon = secondary.attrib['k'].index(":") + 1
        new['key'] = secondary.attrib['k'][post_colon:]
        new['type'] = secondary.attrib['k'][:post_colon - 1]
    new['value'] = secondary.attrib['v']
    return new
    
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    
    if element.tag == "node" or element.tag == "way":
            for tag in element.iter("tag"):
                if is_street_name(tag):
                    tag.attrib['v']=update_name(tag.attrib['v'], mapping)
    
    if element.tag == 'node':
        for attribute, value in element.attrib.iteritems():
            if attribute in node_attr_fields:
                node_attribs[attribute] = value
        
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']) is not None:
                    continue
                else:
                    new_tag = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new_tag)
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attribute, value in element.attrib.iteritems():
            if attribute in way_attr_fields:
                way_attribs[attribute] = value
                
        counter = 0
        for secondary in element.iter():
            if secondary.tag == 'tag':
                if problem_chars.match(secondary.attrib['k']) is not None:
                    continue
                else:
                    new_tag = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new_tag)
            if secondary.tag == 'nd':
                new_node = {}
                new_node['id'] = element.attrib['id']
                new_node['node_id'] = secondary.attrib['ref']
                new_node['position'] = counter
                counter += 1
                way_nodes.append(new_node)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #el = update_name(element, mapping)
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [14]:
data=process_map(dataset, validate=False)

### Taking a look at the csv files

Now we can easily see the content of our csv files using Pandas. To show how easy it is to correct the data in using Pandas we will correct the phone numbers in our dataset.

In [15]:
csv_nodes = pd.read_csv("nodes.csv", encoding="utf-8")
csv_nodes.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,30730952,42.367810,-71.021871,wambag,326503,2,14335103,2012-12-19T19:24:31Z
1,30730953,42.367736,-71.021857,wambag,326503,2,14335103,2012-12-19T19:24:31Z
2,30730954,42.367608,-71.021817,wambag,326503,2,14335103,2012-12-19T19:24:31Z
3,30730955,42.367523,-71.021849,wambag,326503,2,14335103,2012-12-19T19:24:32Z
4,30730956,42.367455,-71.021886,wambag,326503,2,14335103,2012-12-19T19:24:31Z


#### Correcting Phone Numbers with Pandas

In [16]:
csv_nodes_tags = pd.read_csv("nodes_tags.csv", encoding="utf-8")
csv_nodes_tags.head()
csv_nodes_tags.loc[csv_nodes_tags.loc[:,'key'] == "phone",:]

,id,key,value,type
204587,257578009,phone,617-635-8532,regular
204596,257578041,phone,617-484-3519,regular
204605,257578075,phone,617-635-8710,regular
204614,257578119,phone,617-635-8930,regular
204625,257578122,phone,617-635-1534,regular
204635,257578145,phone,617-635-8507,regular
204645,257578151,phone,617-984-8762,regular
204655,257578166,phone,617-552-7800,regular
204666,257578171,phone,(617) 464-2485,regular
204676,257578225,phone,781-326-5350,regular


Here are our helper functions:

In [17]:
def only_digits(string_to_filter):
    """
    Filters the phone number and leaves only digits 0-9
    """
    return ''.join(c for c in string_to_filter if c in "0123456789")

def correct_phone_number(ph_number):
    """
    Transforms the phone number into the ###-###-#### format
    """
    result = only_digits(str(ph_number))
    if len(result) > 0 and result[0] == "1":
        result = result[1:]
    if len(result) == 10:
        return result[0:3] + "-" + result[3:6] + "-" + result[6:10]
    else:
        return "incorrect_num"
num = "+1-617-437-8884"



Here we apply our helper functions to our dataset:

In [18]:
csv_nodes_tags.loc[csv_nodes_tags.loc[:,'key'] == "phone", "value"] = \
csv_nodes_tags.loc[csv_nodes_tags.loc[:,'key'] == "phone", "value"].apply(correct_phone_number)

In [19]:
csv_ways = pd.read_csv("ways.csv", encoding="utf-8")
csv_ways.head()

,id,user,uid,version,changeset,timestamp
0,4790718,JessAk71,381909,3,6963203,2011-01-14T00:06:42Z
1,4790735,claysmalley,119881,4,9069918,2011-08-19T22:12:46Z
2,4824114,effektz,3983283,20,39519438,2016-05-23T20:54:09Z
3,4824115,ezr2,1733549,13,32296955,2015-06-29T23:22:38Z
4,4824116,effektz,3983283,24,39518613,2016-05-23T20:16:38Z


In [20]:
csv_ways_nodes = pd.read_csv("ways_nodes.csv", encoding="utf-8")
csv_ways_nodes.head()

,id,node_id,position
0,4790718,30730967,0
1,4790718,30730968,1
2,4790718,325383304,2
3,4790735,1404985339,0
4,4790735,1404985374,1


In [21]:
csv_ways_tags = pd.read_csv("ways_tags.csv", encoding="utf-8")
csv_ways_tags.head()

,id,key,value,type
0,4790718,foot,yes,regular
1,4790718,name,Skybridge to Hilton,regular
2,4790718,layer,1,regular
3,4790718,bridge,yes,regular
4,4790718,highway,footway,regular


### Insert Data 

We can easily import the data saved in the csv files to the sqlite database using Sqlalchemy package.

In [22]:
disk_engine = sqlalchemy.create_engine('sqlite:///boston_db.db')

In [23]:
csv_nodes.to_sql('nodes', disk_engine, if_exists='replace', index=False)

In [24]:
csv_nodes_tags.to_sql('nodes_tags', disk_engine, if_exists='replace', index=False)

In [25]:
csv_ways.to_sql('ways', disk_engine, if_exists='replace', index=False)

In [26]:
csv_ways_nodes.to_sql('ways_nodes', disk_engine, if_exists='replace', index=False)

In [27]:
csv_ways_tags.to_sql('ways_tags', disk_engine, if_exists='replace', index=False)

## Data Overview

### File size

In [28]:
print "boston_massachusetts.osm: " + str(os.path.getsize(dataset) / 1024 / 1024) + " MB"
print "nodes.csv: " + str(os.path.getsize("nodes.csv") / 1024 / 1024) + " MB"
print "nodes_tags.csv: " + str(os.path.getsize("nodes_tags.csv") / 1024 / 1024) + " MB"
print "ways.csv: " + str(os.path.getsize("ways.csv") / 1024 / 1024) + " MB"
print "ways_nodes.csv: " + str(os.path.getsize("ways_nodes.csv") / 1024 / 1024) + " MB"
print "ways_tags.csv: " + str(os.path.getsize("ways_tags.csv") / 1024 / 1024) + " MB"
print "boston_db.db: " + str(os.path.getsize("boston_db.db") / 1024 / 1024) + " MB"

boston_massachusetts.osm: 413 MB
nodes.csv: 151 MB
nodes_tags.csv: 16 MB
ways.csv: 19 MB
ways_nodes.csv: 52 MB
ways_tags.csv: 21 MB
boston_db.db: 232 MB


### Number of unique users

In [29]:
result = pd.read_sql_query("""
SELECT COUNT(DISTINCT users.uid) AS num_of_unique_users
FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) AS users;
""", disk_engine)
result

,num_of_unique_users
0,1230


### Top 10 contributing users

In [30]:
result = pd.read_sql_query("""
SELECT users.user, COUNT(*) as num_of_contributions
FROM (SELECT user FROM Nodes UNION ALL SELECT user FROM Ways) users
GROUP BY users.user
ORDER BY num_of_contributions DESC
LIMIT 10;
""", disk_engine)
result

,user,num_of_contributions
0,crschmidt,1203569
1,jremillard-massgis,430528
2,OceanVortex,92109
3,wambag,80338
4,morganwahl,69940
5,ryebread,67565
6,MassGIS Import,63340
7,ingalls_imports,32469
8,Ahlzen,27424
9,mapper999,15085


### First contribution

In [31]:
result = pd.read_sql_query("""
SELECT timestamp FROM Nodes UNION SELECT timestamp FROM Ways
ORDER BY timestamp
LIMIT 1;
""", disk_engine)
result

,timestamp
0,2007-06-21T09:47:59Z


### Number of Nodes

In [32]:
result = pd.read_sql_query("""
SELECT COUNT(*) AS number_of_nodes FROM nodes;
""", disk_engine)
result

,number_of_nodes
0,1930240


### Number of ways

In [33]:
result = pd.read_sql_query("""
SELECT COUNT(*) AS number_of_ways FROM ways
""", disk_engine)
result

,number_of_ways
0,308723


### Most popular banks

In [34]:
result = pd.read_sql_query("""
SELECT nodes_tags.value AS bank, 
    COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT id FROM nodes_tags WHERE value='bank') ids
    ON nodes_tags.id=ids.id
WHERE nodes_tags.key='name'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
""", disk_engine)
result

,bank,num
0,Bank of America,12
1,Citizens Bank,10
2,Santander,6
3,Eastern Bank,5
4,TD Bank,5
5,Cambridge Savings Bank,3
6,Cambridge Trust Company,3
7,Citibank,3
8,Brookline Bank,2
9,East Boston Savings Bank,2


### Most popular cousines

In [35]:
result = pd.read_sql_query("""
SELECT nodes_tags.value AS cousine, 
    COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT id FROM nodes_tags WHERE value='restaurant') ids
    ON nodes_tags.id=ids.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
""", disk_engine)
result

,cousine,num
0,pizza,36
1,american,35
2,chinese,29
3,italian,29
4,mexican,26
5,indian,21
6,thai,19
7,asian,13
8,regional,12
9,japanese,11


### Most popular amenities

In [36]:
result = pd.read_sql_query("""
SELECT nodes_tags.value AS amenity, 
    COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT id FROM nodes_tags) ids
    ON nodes_tags.id=ids.id
WHERE nodes_tags.key='amenity'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
""", disk_engine)
result

,amenity,num
0,bench,1060
1,restaurant,592
2,school,513
3,place_of_worship,288
4,library,282
5,bicycle_parking,270
6,cafe,242
7,fast_food,178
8,bicycle_rental,140
9,post_box,114


### Most popular religions

In [37]:
result = pd.read_sql_query("""
SELECT nodes_tags.value AS religion, 
    COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT id FROM nodes_tags WHERE value='place_of_worship') ids
    ON nodes_tags.id=ids.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
""", disk_engine)
result

,religion,num
0,christian,258
1,jewish,10
2,unitarian,3
3,buddhist,1
4,muslim,1
5,unitarian_universalist,1


### Most popular fast-food chains

In [38]:
result = pd.read_sql_query("""
SELECT nodes_tags.value AS chain, 
    COUNT(*) as num
FROM nodes_tags
    JOIN (SELECT DISTINCT id FROM nodes_tags WHERE value='fast_food') ids
    ON nodes_tags.id=ids.id
WHERE nodes_tags.key='name'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
""", disk_engine)
result

,chain,num
0,Dunkin' Donuts,12
1,Subway,10
2,Burger King,7
3,McDonald's,7
4,Wendy's,4
5,Anna's Taqueria,3
6,Chipotle,3
7,Sbarro,3
8,Boloco,2
9,Clover Food Lab,2


### Conclusion          

The Boston OpenStreetMap dataset quite messy. The data contains some mistakes or different references for the same feature (like abbreviation or slightly different names). We had to clean the dataset programmatically for the street names ('Abby Rd' => 'Abby Road', 'Adams St' => 'Adams Street') and phone numbers ('(617) 464-2485' => '617-464-2485', '+1 617 2770 324' => '617-277-0324'). Although is very useful, I learned a few new things about the town I wanted to travel to. 
With this project I learned a lot of new things, took a deep insight into the OpenStreetMaps, which I use quite a long time through MAPS.ME app. Despite the data isn't 100% clean, it's amazing that so many people contribute to the project for free.

We can improve the quality of the OpenStreetMap Data by adding validators at the data insertion stage. It could be implimented in HTML/JavaScript. A good example -- an input tag with a pattern attribute:

<input type="text" id="phonenumber" name="phone" placeholder="555-55-5555" pattern="\d{3}-?\d{2}-?\d{4}">

Pros and Cons of this method:
* (+) Only clean data is being collected, no needs to audit the whole database that I suppose is very large
* (+) Doesn't require extra computing, the validation takes place on the user side. 

* (-) Requires some extra knowledge of HTML/JavaScript
* (-) The format of almost every field depends on the region, which makes it harder to implement
* (-) It doesn't protect the data for example from invalid phone numbers, unexisting places, that contributors can insert, even in a correct format

## References

* http://wiki.openstreetmap.org/wiki/Overpass_API
* http://wiki.openstreetmap.org/wiki/OSM_XML
* https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
* https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/
* http://stackoverflow.com/questions/21650617/html-form-field-how-to-require-an-input-format